# Combine Data.ipynb

Merge all datasets together before conducting analysis.

In [1]:
import pandas as pd


In [2]:
# Concatenate cw_graded.xlsx and gsm8k_graded.xlsx
cw_grading = pd.read_excel('cw_graded.xlsx')
gsm8k_grading = pd.read_excel('gsm8k_graded.xlsx')

stacked_df = pd.concat([cw_grading, gsm8k_grading])
stacked_df


,model_task_method,conversation_number,coherence_1_incoherent_10_very_coherent,task_constraints_followed_0_not_followed_1_followed,ease_of_review_1_easy_10_hard,correct
0,td3_cw_direct_prompting_responses,1,1.0,1.0,1.0,NaN
1,td3_cw_direct_prompting_responses,2,7.0,0.0,1.0,NaN
2,td3_cw_direct_prompting_responses,3,1.0,1.0,1.0,NaN
3,td3_cw_direct_prompting_responses,4,10.0,1.0,1.0,NaN
4,td3_cw_direct_prompting_responses,5,4.0,1.0,1.0,NaN
...,...,...,...,...,...,...
1595,gpt4_gsm8k_manual_cot_responses,96,NaN,NaN,NaN,1.0
1596,gpt4_gsm8k_manual_cot_responses,97,NaN,NaN,NaN,1.0
1597,gpt4_gsm8k_manual_cot_responses,98,NaN,NaN,NaN,1.0
1598,gpt4_gsm8k_manual_cot_responses,99,NaN,NaN,NaN,1.0


### Adding fine-tuned GPT-3.5 predictions

In [3]:
# Predictions from fine-tuned GPT-3.5
# Merge on "predictions_based_on_first_10.csv", "items_to_predict_using_last_10.csv"
# Limit columns to model_task_method, conversation_number, Prediction

first_10 = pd.read_csv('predictions_based_on_first_10.csv')[['model_task_method', 'conversation_number', 'Prediction']]
# Rename column Prediction to Prediction_Based_On_First_10
first_10.rename(columns={'Prediction': 'Prediction_Based_On_First_10'}, inplace=True)

last_10 = pd.read_csv('predictions_based_on_last_10.csv')[['model_task_method', 'conversation_number', 'Prediction']]
# Rename column Prediction to Prediction_Based_On_Last_10
last_10.rename(columns={'Prediction': 'Prediction_Based_On_Last_10'}, inplace=True)

print(first_10)
print(last_10)


                      model_task_method  conversation_number  \
0     td3_cw_direct_prompting_responses                   11   
1     td3_cw_direct_prompting_responses                   12   
2     td3_cw_direct_prompting_responses                   13   
3     td3_cw_direct_prompting_responses                   14   
4     td3_cw_direct_prompting_responses                   15   
...                                 ...                  ...   
1435       gpt4_cw_manual_cot_responses                   96   
1436       gpt4_cw_manual_cot_responses                   97   
1437       gpt4_cw_manual_cot_responses                   98   
1438       gpt4_cw_manual_cot_responses                   99   
1439       gpt4_cw_manual_cot_responses                  100   

     Prediction_Based_On_First_10  
0                  {"Score": "7"}  
1                  {"Score": "7"}  
2                  {"Score": "3"}  
3                  {"Score": "8"}  
4                  {"Score": "3"}  
...            

In [4]:
# Merge on model_task_method and conversation_number
stacked_df_w_pred = stacked_df.merge(first_10, on=['model_task_method', 'conversation_number'], how = 'left')
stacked_df_w_pred = stacked_df_w_pred.merge(last_10, on=['model_task_method', 'conversation_number'], how = 'left')


In [5]:
# Remove {"Score": "
# Remove "}
stacked_df_w_pred['Prediction_Based_On_First_10'] = stacked_df_w_pred['Prediction_Based_On_First_10'].str.replace('{"Score": "', '')
stacked_df_w_pred['Prediction_Based_On_First_10'] = stacked_df_w_pred['Prediction_Based_On_First_10'].str.replace('"}', '')
stacked_df_w_pred['Prediction_Based_On_Last_10'] = stacked_df_w_pred['Prediction_Based_On_Last_10'].str.replace('{"Score": "', '')
stacked_df_w_pred['Prediction_Based_On_Last_10'] = stacked_df_w_pred['Prediction_Based_On_Last_10'].str.replace('"}', '')

# Convert score to float
stacked_df_w_pred['Prediction_Based_On_First_10'] = stacked_df_w_pred['Prediction_Based_On_First_10'].astype(float)
stacked_df_w_pred['Prediction_Based_On_Last_10'] = stacked_df_w_pred['Prediction_Based_On_Last_10'].astype(float)

stacked_df_w_pred


,model_task_method,conversation_number,coherence_1_incoherent_10_very_coherent,task_constraints_followed_0_not_followed_1_followed,ease_of_review_1_easy_10_hard,correct,Prediction_Based_On_First_10,Prediction_Based_On_Last_10
0,td3_cw_direct_prompting_responses,1,1.0,1.0,1.0,NaN,NaN,1.0
1,td3_cw_direct_prompting_responses,2,7.0,0.0,1.0,NaN,NaN,7.0
2,td3_cw_direct_prompting_responses,3,1.0,1.0,1.0,NaN,NaN,1.0
3,td3_cw_direct_prompting_responses,4,10.0,1.0,1.0,NaN,NaN,7.0
4,td3_cw_direct_prompting_responses,5,4.0,1.0,1.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...
3195,gpt4_gsm8k_manual_cot_responses,96,NaN,NaN,NaN,1.0,NaN,NaN
3196,gpt4_gsm8k_manual_cot_responses,97,NaN,NaN,NaN,1.0,NaN,NaN
3197,gpt4_gsm8k_manual_cot_responses,98,NaN,NaN,NaN,1.0,NaN,NaN
3198,gpt4_gsm8k_manual_cot_responses,99,NaN,NaN,NaN,1.0,NaN,NaN


In [6]:
# Create column "Aggregated_Prediction" as the mean of the two predictions
# If one is missing, use the other
stacked_df_w_pred['Aggregated_Prediction'] = stacked_df_w_pred[['Prediction_Based_On_First_10', 'Prediction_Based_On_Last_10']].mean(axis=1)
stacked_df_w_pred['Aggregated_Prediction'] = stacked_df_w_pred['Aggregated_Prediction'].fillna(stacked_df_w_pred['Prediction_Based_On_First_10'])
stacked_df_w_pred['Aggregated_Prediction'] = stacked_df_w_pred['Aggregated_Prediction'].fillna(stacked_df_w_pred['Prediction_Based_On_Last_10'])


#### Finetuning with longer prompts

In [7]:
# Predictions from fine-tuned GPT-3.5
# Merge on "predictions_based_on_first_10.csv", "items_to_predict_using_last_10.csv"
# Limit columns to model_task_method, conversation_number, Prediction

first_10_LP = pd.read_csv('predictions_based_on_first_10_LP.csv')[['model_task_method', 'conversation_number', 'Prediction', 'response']]
# Rename column Prediction to Prediction_Based_On_First_10
first_10_LP.rename(columns={'Prediction': 'Prediction_Based_On_First_10_LP'}, inplace=True)
# Rename column response
first_10_LP.rename(columns={'response': 'response_Based_On_First_10_LP'}, inplace=True)

last_10_LP = pd.read_csv('predictions_based_on_last_10_LP.csv')[['model_task_method', 'conversation_number', 'Prediction', 'response']]
# Rename column Prediction to Prediction_Based_On_Last_10
last_10_LP.rename(columns={'Prediction': 'Prediction_Based_On_Last_10_LP'}, inplace=True)
# Rename column response
last_10_LP.rename(columns={'response': 'response_Based_On_Last_10_LP'}, inplace=True)

print(first_10_LP)
print(last_10_LP)


                      model_task_method  conversation_number  \
0     td3_cw_direct_prompting_responses                   11   
1     td3_cw_direct_prompting_responses                   12   
2     td3_cw_direct_prompting_responses                   13   
3     td3_cw_direct_prompting_responses                   14   
4     td3_cw_direct_prompting_responses                   15   
...                                 ...                  ...   
1435       gpt4_cw_manual_cot_responses                   96   
1436       gpt4_cw_manual_cot_responses                   97   
1437       gpt4_cw_manual_cot_responses                   98   
1438       gpt4_cw_manual_cot_responses                   99   
1439       gpt4_cw_manual_cot_responses                  100   

     Prediction_Based_On_First_10_LP  \
0                     {"Score": "7"}   
1                     {"Score": "7"}   
2                     {"Score": "3"}   
3                     {"Score": "8"}   
4                     {"Score":

In [8]:
# Merge on model_task_method and conversation_number
stacked_df_w_pred = stacked_df_w_pred.merge(first_10_LP, on=['model_task_method', 'conversation_number'], how = 'left')
stacked_df_w_pred = stacked_df_w_pred.merge(last_10_LP, on=['model_task_method', 'conversation_number'], how = 'left')


In [9]:
# Create response column based on response_Based_On_First_10_LP and response_Based_On_Last_10_LP
# Use response_Based_On_First_10_LP if available, otherwise use response_Based_On_Last_10_LP
stacked_df_w_pred['response_LP'] = stacked_df_w_pred['response_Based_On_First_10_LP'].fillna(stacked_df_w_pred['response_Based_On_Last_10_LP'])


In [10]:
# Remove {"Score": "
# Remove "}
stacked_df_w_pred['Prediction_Based_On_First_10_LP'] = stacked_df_w_pred['Prediction_Based_On_First_10_LP'].str.replace('{"Score": "', '')
stacked_df_w_pred['Prediction_Based_On_First_10_LP'] = stacked_df_w_pred['Prediction_Based_On_First_10_LP'].str.replace('"}', '')
stacked_df_w_pred['Prediction_Based_On_Last_10_LP'] = stacked_df_w_pred['Prediction_Based_On_Last_10_LP'].str.replace('{"Score": "', '')
stacked_df_w_pred['Prediction_Based_On_Last_10_LP'] = stacked_df_w_pred['Prediction_Based_On_Last_10_LP'].str.replace('"}', '')

# Convert score to float
stacked_df_w_pred['Prediction_Based_On_First_10_LP'] = stacked_df_w_pred['Prediction_Based_On_First_10_LP'].astype(float)
stacked_df_w_pred['Prediction_Based_On_Last_10_LP'] = stacked_df_w_pred['Prediction_Based_On_Last_10_LP'].astype(float)

stacked_df_w_pred


,model_task_method,conversation_number,coherence_1_incoherent_10_very_coherent,task_constraints_followed_0_not_followed_1_followed,ease_of_review_1_easy_10_hard,correct,Prediction_Based_On_First_10,Prediction_Based_On_Last_10,Aggregated_Prediction,Prediction_Based_On_First_10_LP,response_Based_On_First_10_LP,Prediction_Based_On_Last_10_LP,response_Based_On_Last_10_LP,response_LP
0,td3_cw_direct_prompting_responses,1,1.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,Learning to do a handstand is a fun activity f...,Learning to do a handstand is a fun activity f...
1,td3_cw_direct_prompting_responses,2,7.0,0.0,1.0,NaN,NaN,7.0,7.0,NaN,NaN,6.0,The hawk was used to hunting what he needed fo...,The hawk was used to hunting what he needed fo...
2,td3_cw_direct_prompting_responses,3,1.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,2.0,I love the smell of roasting almonds in the ki...,I love the smell of roasting almonds in the ki...
3,td3_cw_direct_prompting_responses,4,10.0,1.0,1.0,NaN,NaN,7.0,7.0,NaN,NaN,8.0,Ralph's bedroom was routinely filled up with s...,Ralph's bedroom was routinely filled up with s...
4,td3_cw_direct_prompting_responses,5,4.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,2.0,Joe had a unique way of dealing with the hospi...,Joe had a unique way of dealing with the hospi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,gpt4_gsm8k_manual_cot_responses,96,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3196,gpt4_gsm8k_manual_cot_responses,97,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3197,gpt4_gsm8k_manual_cot_responses,98,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3198,gpt4_gsm8k_manual_cot_responses,99,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Create column "Aggregated_Prediction" as the mean of the two predictions
# If one is missing, use the other
stacked_df_w_pred['Aggregated_Prediction_LP'] = stacked_df_w_pred[['Prediction_Based_On_First_10_LP', 'Prediction_Based_On_Last_10_LP']].mean(axis=1)
stacked_df_w_pred['Aggregated_Prediction_LP'] = stacked_df_w_pred['Aggregated_Prediction_LP'].fillna(stacked_df_w_pred['Prediction_Based_On_First_10_LP'])
stacked_df_w_pred['Aggregated_Prediction_LP'] = stacked_df_w_pred['Aggregated_Prediction_LP'].fillna(stacked_df_w_pred['Prediction_Based_On_Last_10_LP'])


### More finetuned predictions

First-last split

In [12]:
# Predictions from fine-tuned GPT-3.5
# Limit columns to model_task_method, conversation_number, Prediction

first_50_LP = pd.read_csv('predictions_based_on_first_50_LP.csv')[['model_task_method', 'conversation_number', 'Prediction', 'response']]
# Rename column Prediction to Prediction_Based_On_First_10
first_50_LP.rename(columns={'Prediction': 'Prediction_Based_On_First_50_LP'}, inplace=True)
# Rename column response
first_50_LP.rename(columns={'response': 'response_Based_On_First_50_LP'}, inplace=True)

last_50_LP = pd.read_csv('predictions_based_on_last_50_LP.csv')[['model_task_method', 'conversation_number', 'Prediction', 'response']]
# Rename column Prediction to Prediction_Based_On_Last_10
last_50_LP.rename(columns={'Prediction': 'Prediction_Based_On_Last_50_LP'}, inplace=True)
# Rename column response
last_50_LP.rename(columns={'response': 'response_Based_On_Last_50_LP'}, inplace=True)

# Merge on model_task_method and conversation_number
stacked_df_w_pred = stacked_df_w_pred.merge(first_50_LP, on=['model_task_method', 'conversation_number'], how = 'left')
stacked_df_w_pred = stacked_df_w_pred.merge(last_50_LP, on=['model_task_method', 'conversation_number'], how = 'left')

# Create response column based on response_Based_On_First_10_LP and response_Based_On_Last_10_LP
# Use response_Based_On_First_10_LP if available, otherwise use response_Based_On_Last_10_LP
# stacked_df_w_pred['response_LP'] = stacked_df_w_pred['response_Based_On_First_50_LP'].fillna(stacked_df_w_pred['response_Based_On_Last_50_LP'])

# Remove {"Score": "
# Remove "}
stacked_df_w_pred['Prediction_Based_On_First_50_LP'] = stacked_df_w_pred['Prediction_Based_On_First_50_LP'].str.replace('{"Score": "', '')
stacked_df_w_pred['Prediction_Based_On_First_50_LP'] = stacked_df_w_pred['Prediction_Based_On_First_50_LP'].str.replace('"}', '')
stacked_df_w_pred['Prediction_Based_On_Last_50_LP'] = stacked_df_w_pred['Prediction_Based_On_Last_50_LP'].str.replace('{"Score": "', '')
stacked_df_w_pred['Prediction_Based_On_Last_50_LP'] = stacked_df_w_pred['Prediction_Based_On_Last_50_LP'].str.replace('"}', '')

# Convert score to float
stacked_df_w_pred['Prediction_Based_On_First_50_LP'] = stacked_df_w_pred['Prediction_Based_On_First_50_LP'].astype(float)
stacked_df_w_pred['Prediction_Based_On_Last_50_LP'] = stacked_df_w_pred['Prediction_Based_On_Last_50_LP'].astype(float)

# Create column "Aggregated_Prediction" as the mean of the two predictions
# If one is missing, use the other
stacked_df_w_pred['Aggregated_Prediction_50_LP'] = stacked_df_w_pred[['Prediction_Based_On_First_50_LP', 'Prediction_Based_On_Last_50_LP']].mean(axis=1)
stacked_df_w_pred['Aggregated_Prediction_50_LP'] = stacked_df_w_pred['Aggregated_Prediction_50_LP'].fillna(stacked_df_w_pred['Prediction_Based_On_First_50_LP'])
stacked_df_w_pred['Aggregated_Prediction_50_LP'] = stacked_df_w_pred['Aggregated_Prediction_50_LP'].fillna(stacked_df_w_pred['Prediction_Based_On_Last_50_LP'])


In [13]:
print(stacked_df_w_pred.columns)


Index(['model_task_method', 'conversation_number',
       'coherence_1_incoherent_10_very_coherent',
       'task_constraints_followed_0_not_followed_1_followed',
       'ease_of_review_1_easy_10_hard', 'correct',
       'Prediction_Based_On_First_10', 'Prediction_Based_On_Last_10',
       'Aggregated_Prediction', 'Prediction_Based_On_First_10_LP',
       'response_Based_On_First_10_LP', 'Prediction_Based_On_Last_10_LP',
       'response_Based_On_Last_10_LP', 'response_LP',
       'Aggregated_Prediction_LP', 'Prediction_Based_On_First_50_LP',
       'response_Based_On_First_50_LP', 'Prediction_Based_On_Last_50_LP',
       'response_Based_On_Last_50_LP', 'Aggregated_Prediction_50_LP'],
      dtype='object')


Random split

In [14]:
random_50_LP_1 = pd.read_csv('predictions_based_on_random_50_LP_1.csv')[['model_task_method', 'conversation_number', 'Prediction', 'response']]
# Rename column Prediction to Prediction_Based_On_random_50_LP_1
random_50_LP_1.rename(columns={'Prediction': 'Prediction_Based_On_random_50_LP_1'}, inplace=True)
# Rename column response
random_50_LP_1.rename(columns={'response': 'response_Based_On_random_50_LP_1'}, inplace=True)

random_50_LP_2 = pd.read_csv('predictions_based_on_random_50_LP_2.csv')[['model_task_method', 'conversation_number', 'Prediction', 'response']]
# Rename column Prediction to Prediction_Based_On_random_50_LP_2
random_50_LP_2.rename(columns={'Prediction': 'Prediction_Based_On_random_50_LP_2'}, inplace=True)
# Rename column response
random_50_LP_2.rename(columns={'response': 'response_Based_On_random_50_LP_2'}, inplace=True)

# Merge on model_task_method and conversation_number
stacked_df_w_pred = stacked_df_w_pred.merge(random_50_LP_1, on=['model_task_method', 'conversation_number'], how = 'left')
stacked_df_w_pred = stacked_df_w_pred.merge(random_50_LP_2, on=['model_task_method', 'conversation_number'], how = 'left')

# Create response column based on response_Based_On_First_10_LP and response_Based_On_Last_10_LP
# Use response_Based_On_First_10_LP if available, otherwise use response_Based_On_Last_10_LP
# stacked_df_w_pred['response_LP'] = stacked_df_w_pred['response_Based_On_First_50_LP'].fillna(stacked_df_w_pred['response_Based_On_Last_50_LP'])

# Remove {"Score": "
# Remove "}
stacked_df_w_pred['Prediction_Based_On_random_50_LP_1'] = stacked_df_w_pred['Prediction_Based_On_random_50_LP_1'].str.replace('{"Score": "', '')
stacked_df_w_pred['Prediction_Based_On_random_50_LP_1'] = stacked_df_w_pred['Prediction_Based_On_random_50_LP_1'].str.replace('"}', '')
stacked_df_w_pred['Prediction_Based_On_random_50_LP_2'] = stacked_df_w_pred['Prediction_Based_On_random_50_LP_2'].str.replace('{"Score": "', '')
stacked_df_w_pred['Prediction_Based_On_random_50_LP_2'] = stacked_df_w_pred['Prediction_Based_On_random_50_LP_2'].str.replace('"}', '')

# Convert score to float
stacked_df_w_pred['Prediction_Based_On_random_50_LP_1'] = stacked_df_w_pred['Prediction_Based_On_random_50_LP_1'].astype(float)
stacked_df_w_pred['Prediction_Based_On_random_50_LP_2'] = stacked_df_w_pred['Prediction_Based_On_random_50_LP_2'].astype(float)

# Create column "Aggregated_Prediction" as the mean of the two predictions
# If one is missing, use the other
stacked_df_w_pred['Aggregated_Prediction_random_50_LP'] = stacked_df_w_pred[['Prediction_Based_On_random_50_LP_1', 'Prediction_Based_On_random_50_LP_2']].mean(axis=1)
stacked_df_w_pred['Aggregated_Prediction_random_50_LP'] = stacked_df_w_pred['Aggregated_Prediction_random_50_LP'].fillna(stacked_df_w_pred['Prediction_Based_On_random_50_LP_1'])
stacked_df_w_pred['Aggregated_Prediction_random_50_LP'] = stacked_df_w_pred['Aggregated_Prediction_random_50_LP'].fillna(stacked_df_w_pred['Prediction_Based_On_random_50_LP_2'])


### Sentence-BERT Local Coherence

In [15]:
# Load actual_cw_passage_content_with_sim.xlsx
actual_cw_passage_content_with_sim = pd.read_excel('actual_cw_passage_content_with_sim.xlsx')
print(actual_cw_passage_content_with_sim.columns)

# Merge on model_task_method and conversation_number
stacked_df_w_pred = stacked_df_w_pred.merge(actual_cw_passage_content_with_sim, on=['model_task_method', 'conversation_number'], how = 'left')
print(stacked_df_w_pred.columns)


Index(['Unnamed: 0', 'model_task_method', 'conversation_number', 'response',
       'replace_slash_n_slash_n_with_newline',
       'replace_slash_n_slash_n_with_newline_values',
       'replace_slash_n_with_newline', 'replace_slash_n_with_newline_values',
       'avg_cosine_sim', 'num_sentences'],
      dtype='object')
Index(['model_task_method', 'conversation_number',
       'coherence_1_incoherent_10_very_coherent',
       'task_constraints_followed_0_not_followed_1_followed',
       'ease_of_review_1_easy_10_hard', 'correct',
       'Prediction_Based_On_First_10', 'Prediction_Based_On_Last_10',
       'Aggregated_Prediction', 'Prediction_Based_On_First_10_LP',
       'response_Based_On_First_10_LP', 'Prediction_Based_On_Last_10_LP',
       'response_Based_On_Last_10_LP', 'response_LP',
       'Aggregated_Prediction_LP', 'Prediction_Based_On_First_50_LP',
       'response_Based_On_First_50_LP', 'Prediction_Based_On_Last_50_LP',
       'response_Based_On_Last_50_LP', 'Aggregated_Predi

### Sentence-BERT Inter-Paragraph Coherence

In [16]:
# Load actual_cw_passage_content_with_inter_paragraph_sim.xlsx
actual_cw_passage_content_with_inter_paragraph_sim = pd.read_excel('actual_cw_passage_content_with_inter_paragraph_sim.xlsx')
print(actual_cw_passage_content_with_inter_paragraph_sim.columns)

# Merge on model_task_method and conversation_number
stacked_df_w_pred = stacked_df_w_pred.merge(actual_cw_passage_content_with_inter_paragraph_sim, on=['model_task_method', 'conversation_number'], how = 'left')
print(stacked_df_w_pred.columns)


Index(['Unnamed: 0', 'model_task_method', 'conversation_number', 'response',
       'replace_slash_n_slash_n_with_newline',
       'replace_slash_n_slash_n_with_newline_values',
       'replace_slash_n_with_newline', 'replace_slash_n_with_newline_values',
       'avg_inter_paragraph_cosine_sim', 'num_paragraphs', 'num_sentences',
       'cosine_sims'],
      dtype='object')
Index(['model_task_method', 'conversation_number',
       'coherence_1_incoherent_10_very_coherent',
       'task_constraints_followed_0_not_followed_1_followed',
       'ease_of_review_1_easy_10_hard', 'correct',
       'Prediction_Based_On_First_10', 'Prediction_Based_On_Last_10',
       'Aggregated_Prediction', 'Prediction_Based_On_First_10_LP',
       'response_Based_On_First_10_LP', 'Prediction_Based_On_Last_10_LP',
       'response_Based_On_Last_10_LP', 'response_LP',
       'Aggregated_Prediction_LP', 'Prediction_Based_On_First_50_LP',
       'response_Based_On_First_50_LP', 'Prediction_Based_On_Last_50_LP',


### Back to more metrics

In [17]:
# Automatic_Metrics.xlsx
Automatic_Metrics = pd.read_excel('Automatic_Metrics.xlsx')

Automatic_Metrics


,model_task_method,conversation_number,conversation_length,input_length,output_length,conversation_cost,gsm8k_question_index,gsm8k_answer,gsm8k_length_vs_provided,length_vs_direct_prompting,...,num_sentences_prompts,num_step_i_prompts,num_1_dot_etc_prompts,sentence_length_prompts,fres_prompts,num_linebreaks_provided,num_sentences_provided,num_step_i_provided,num_1_dot_etc_provided,length_provided
0,td3_gsm8k_direct_prompting_responses,1,82,69,12,0.00164,0.0,Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eg...,0.689076,1.000000,...,5,0,0,NaN,NaN,2.0,3.0,0.0,0.0,53.0
1,td3_gsm8k_direct_prompting_responses,2,36,30,5,0.00072,1.0,It takes 2/2=<<2/2=1>>1 bolt of white fiber\nS...,0.514286,1.000000,...,3,0,0,NaN,NaN,2.0,1.0,0.0,0.0,44.0
2,td3_gsm8k_direct_prompting_responses,3,104,53,50,0.00208,2.0,The cost of the house and repairs came out to ...,0.608187,1.000000,...,5,0,0,NaN,NaN,4.0,1.0,0.0,2.0,122.0
3,td3_gsm8k_direct_prompting_responses,4,39,36,2,0.00078,3.0,He sprints 3*3=<<3*3=9>>9 times\nSo he runs 9*...,0.573529,1.000000,...,4,0,0,NaN,NaN,2.0,1.0,0.0,0.0,36.0
4,td3_gsm8k_direct_prompting_responses,5,115,109,5,0.00230,4.0,"If each chicken eats 3 cups of feed per day, t...",0.598958,1.000000,...,6,0,0,NaN,NaN,2.0,3.0,0.0,0.0,87.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,gpt4_cw_manual_cot_responses,96,949,704,245,0.03582,NaN,NaN,NaN,2.711429,...,35,0,12,15.257143,73.78,NaN,NaN,NaN,NaN,NaN
3196,gpt4_cw_manual_cot_responses,97,1012,714,298,0.03930,NaN,NaN,NaN,2.899713,...,35,0,12,15.485714,73.58,NaN,NaN,NaN,NaN,NaN
3197,gpt4_cw_manual_cot_responses,98,979,711,268,0.03741,NaN,NaN,NaN,2.734637,...,35,0,12,15.428571,73.58,NaN,NaN,NaN,NaN,NaN
3198,gpt4_cw_manual_cot_responses,99,1023,703,320,0.04029,NaN,NaN,NaN,3.177019,...,35,0,12,15.228571,73.78,NaN,NaN,NaN,NaN,NaN


In [18]:
# Merge on model_task_method, conversation_number
merged_df = pd.merge(stacked_df_w_pred, Automatic_Metrics, on=['model_task_method', 'conversation_number'], how='left')

merged_df


,model_task_method,conversation_number,coherence_1_incoherent_10_very_coherent,task_constraints_followed_0_not_followed_1_followed,ease_of_review_1_easy_10_hard,correct,Prediction_Based_On_First_10,Prediction_Based_On_Last_10,Aggregated_Prediction,Prediction_Based_On_First_10_LP,...,num_sentences_prompts,num_step_i_prompts,num_1_dot_etc_prompts,sentence_length_prompts,fres_prompts,num_linebreaks_provided,num_sentences_provided,num_step_i_provided,num_1_dot_etc_provided,length_provided
0,td3_cw_direct_prompting_responses,1,1.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,...,5,0,2,8.6,94.35,NaN,NaN,NaN,NaN,NaN
1,td3_cw_direct_prompting_responses,2,7.0,0.0,1.0,NaN,NaN,7.0,7.0,NaN,...,5,0,2,8.4,86.20,NaN,NaN,NaN,NaN,NaN
2,td3_cw_direct_prompting_responses,3,1.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,...,5,0,2,10.8,74.69,NaN,NaN,NaN,NaN,NaN
3,td3_cw_direct_prompting_responses,4,10.0,1.0,1.0,NaN,NaN,7.0,7.0,NaN,...,5,0,2,7.8,69.99,NaN,NaN,NaN,NaN,NaN
4,td3_cw_direct_prompting_responses,5,4.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,...,5,0,2,10.6,74.90,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,gpt4_gsm8k_manual_cot_responses,96,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,45,0,20,NaN,NaN,4.0,5.0,0.0,1.0,103.0
3196,gpt4_gsm8k_manual_cot_responses,97,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,46,0,20,NaN,NaN,2.0,3.0,0.0,0.0,49.0
3197,gpt4_gsm8k_manual_cot_responses,98,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,47,0,20,NaN,NaN,3.0,4.0,0.0,0.0,90.0
3198,gpt4_gsm8k_manual_cot_responses,99,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,47,0,20,NaN,NaN,2.0,3.0,0.0,0.0,94.0


### CW Compliance

In [19]:
# Add Creative_Writing_Compliance.xlsx
Creative_Writing_Compliance = pd.read_excel('Creative_Writing_Compliance.xlsx')

merged_df = pd.merge(merged_df, Creative_Writing_Compliance, on=['model_task_method', 'conversation_number'], how='left')
merged_df


,model_task_method,conversation_number,coherence_1_incoherent_10_very_coherent,task_constraints_followed_0_not_followed_1_followed,ease_of_review_1_easy_10_hard,correct,Prediction_Based_On_First_10,Prediction_Based_On_Last_10,Aggregated_Prediction,Prediction_Based_On_First_10_LP,...,num_step_i_prompts,num_1_dot_etc_prompts,sentence_length_prompts,fres_prompts,num_linebreaks_provided,num_sentences_provided,num_step_i_provided,num_1_dot_etc_provided,length_provided,compliance
0,td3_cw_direct_prompting_responses,1,1.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,...,0,2,8.6,94.35,NaN,NaN,NaN,NaN,NaN,1.0
1,td3_cw_direct_prompting_responses,2,7.0,0.0,1.0,NaN,NaN,7.0,7.0,NaN,...,0,2,8.4,86.20,NaN,NaN,NaN,NaN,NaN,0.0
2,td3_cw_direct_prompting_responses,3,1.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,...,0,2,10.8,74.69,NaN,NaN,NaN,NaN,NaN,1.0
3,td3_cw_direct_prompting_responses,4,10.0,1.0,1.0,NaN,NaN,7.0,7.0,NaN,...,0,2,7.8,69.99,NaN,NaN,NaN,NaN,NaN,1.0
4,td3_cw_direct_prompting_responses,5,4.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,...,0,2,10.6,74.90,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,gpt4_gsm8k_manual_cot_responses,96,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,0,20,NaN,NaN,4.0,5.0,0.0,1.0,103.0,NaN
3196,gpt4_gsm8k_manual_cot_responses,97,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,0,20,NaN,NaN,2.0,3.0,0.0,0.0,49.0,NaN
3197,gpt4_gsm8k_manual_cot_responses,98,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,0,20,NaN,NaN,3.0,4.0,0.0,0.0,90.0,NaN
3198,gpt4_gsm8k_manual_cot_responses,99,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,0,20,NaN,NaN,2.0,3.0,0.0,0.0,94.0,NaN


In [20]:
# Adjust compliance based on num_paragraphs
# Set to zero if num_paragraphs is not 2
# Mean of compliance beforehand
print(merged_df['compliance'].mean())
# Make change
merged_df['compliance'] = merged_df['compliance'] * (merged_df['num_paragraphs'] == 2)
# Mean of compliance afterwards
print(merged_df['compliance'].mean())


0.4275
0.418125


In [21]:
# Check for missing values of compliance
merged_df['compliance'].isna().sum()


1600

In [22]:
# Create coherence_1_incoherent_10_very_coherent_compliance_adjusted
# This is coherence_1_incoherent_10_very_coherent but if the conversation is not compliant, the score is 1
merged_df['coherence_1_incoherent_10_very_coherent_compliance_adjusted'] = merged_df['coherence_1_incoherent_10_very_coherent']
merged_df.loc[merged_df['compliance'] == 0, 'coherence_1_incoherent_10_very_coherent_compliance_adjusted'] = 1


In [23]:
# Aggregated prediction, compliance adjusted
merged_df['Aggregated_Prediction_random_50_LP_compliance_adjusted'] = merged_df['Aggregated_Prediction_random_50_LP']
merged_df.loc[merged_df['compliance'] == 0, 'Aggregated_Prediction_random_50_LP_compliance_adjusted'] = 1


In [24]:
# Cosine similarities, compliance adjusted
# Just set these to NA
merged_df['avg_cosine_sim_compliance_adjusted'] = merged_df['avg_cosine_sim']
merged_df['avg_inter_paragraph_cosine_sim_compliance_adjusted'] = merged_df['avg_inter_paragraph_cosine_sim']
merged_df.loc[merged_df['compliance'] == 0, 'avg_cosine_sim_compliance_adjusted'] = None
merged_df.loc[merged_df['compliance'] == 0, 'avg_inter_paragraph_cosine_sim_compliance_adjusted'] = None


In [25]:
# Rename old compliance column
merged_df.rename(columns={'task_constraints_followed_0_not_followed_1_followed': 'compliance_OLD'}, inplace=True)


## Adding Columns

In [26]:
# Get model, task, and method from model_task_method
# Model - first item when split on underscores
# Task - second item when split on underscores
# method - all remaining items when split on underscores, removing the last item
merged_df['model'] = merged_df['model_task_method'].str.split('_').str[0]
merged_df['task'] = merged_df['model_task_method'].str.split('_').str[1]
merged_df['method'] = merged_df['model_task_method'].str.split('_').str[2:].str[:-1].str.join('_')

# Print value counts of model, task, and method
print(merged_df['model'].value_counts())
print(merged_df['task'].value_counts())
print(merged_df['method'].value_counts())


model
td3     1600
gpt4    1600
Name: count, dtype: int64
task
cw       1600
gsm8k    1600
Name: count, dtype: int64
method
direct_prompting     400
zero_shot_cot        400
ape_zero_shot_cot    400
least_to_most        400
manual_few_shot      400
manual_cot           400
tree_of_thought      400
self_refine          400
Name: count, dtype: int64


In [27]:
# Cleaned up strings
# Recode 'model' as 'Model' - change gpt4 to "GPT-4" and td3 to "Text-Davinci-003"
merged_df['Model'] = merged_df['model'].replace({'gpt4': 'GPT-4', 'td3': 'Text-Davinci-003'})
# Recode 'method' as 'Method' - change ape_zero_shot_cot to "APE Zero-Shot COT", zero_shot_cot to "Zero-Shot COT", least_to_most to "Least-to-Most", manual_cot to "Manual COT", manual_few_shot to "Manual Few-Shot", self_refine to "Self-Refine", tree_of_thought to "Tree-of-Thought"
merged_df['Method'] = merged_df['method'].replace({'ape_zero_shot_cot': 'APE Zero-Shot CoT', 'zero_shot_cot': 'Zero-Shot CoT', 'least_to_most': 'Least-to-Most', 'manual_cot': 'Manual CoT', 'manual_few_shot': 'Manual Few-Shot', 'self_refine': 'Self-Refine', 'tree_of_thought': 'Tree-of-Thought', 'direct_prompting' : 'Direct Prompting'})
# Create 'Task' - change cw to "Creative Writing", gsm8k to "GSM8K"
merged_df['Task'] = merged_df['task'].replace({'cw': 'Creative Writing', 'gsm8k': 'GSM8K'})


In [28]:
print(merged_df['Model'].value_counts())
print(merged_df['Method'].value_counts())


Model
Text-Davinci-003    1600
GPT-4               1600
Name: count, dtype: int64
Method
Direct Prompting     400
Zero-Shot CoT        400
APE Zero-Shot CoT    400
Least-to-Most        400
Manual Few-Shot      400
Manual CoT           400
Tree-of-Thought      400
Self-Refine          400
Name: count, dtype: int64


In [29]:
# Column for combined accuracy_quality - correct_or_incorrect if task = "gsm8k", creative_writing_score if task = "cw"
merged_df['accuracy_quality'] = merged_df.apply(lambda row: row['correct'] if row['task'] == 'gsm8k' else row['coherence_1_incoherent_10_very_coherent'], axis=1)

# Also create accuracy_quality_compliance_adjusted
merged_df['accuracy_quality_compliance_adjusted'] = merged_df.apply(lambda row: row['correct'] if row['task'] == 'gsm8k' else row['coherence_1_incoherent_10_very_coherent_compliance_adjusted'], axis=1)

# Also create accuracy_quality_avg_inter_paragraph_cosine_sim
# This is preferred for gains per token metrics
merged_df['accuracy_quality_avg_inter_paragraph_cosine_sim'] = merged_df.apply(lambda row: row['correct'] if row['task'] == 'gsm8k' else row['avg_inter_paragraph_cosine_sim'], axis=1)


### Consolidated Number of Steps

In [30]:
print(merged_df.columns)
# Sum num_step_i and num_1_dot_etc
merged_df['consolidated_num_steps_ideas'] = merged_df['num_step_i'] + merged_df['num_1_dot_etc']
# Also do for _prompts and _provided
merged_df['consolidated_num_steps_ideas_prompts'] = merged_df['num_step_i_prompts'] + merged_df['num_1_dot_etc_prompts']
merged_df['consolidated_num_steps_ideas_provided'] = merged_df['num_step_i_provided'] + merged_df['num_1_dot_etc_provided']


Index(['model_task_method', 'conversation_number',
       'coherence_1_incoherent_10_very_coherent', 'compliance_OLD',
       'ease_of_review_1_easy_10_hard', 'correct',
       'Prediction_Based_On_First_10', 'Prediction_Based_On_Last_10',
       'Aggregated_Prediction', 'Prediction_Based_On_First_10_LP',
       'response_Based_On_First_10_LP', 'Prediction_Based_On_Last_10_LP',
       'response_Based_On_Last_10_LP', 'response_LP',
       'Aggregated_Prediction_LP', 'Prediction_Based_On_First_50_LP',
       'response_Based_On_First_50_LP', 'Prediction_Based_On_Last_50_LP',
       'response_Based_On_Last_50_LP', 'Aggregated_Prediction_50_LP',
       'Prediction_Based_On_random_50_LP_1',
       'response_Based_On_random_50_LP_1',
       'Prediction_Based_On_random_50_LP_2',
       'response_Based_On_random_50_LP_2',
       'Aggregated_Prediction_random_50_LP', 'Unnamed: 0_x', 'response_x',
       'replace_slash_n_slash_n_with_newline_x',
       'replace_slash_n_slash_n_with_newline_values

### Add Method SS Publication Date

In [31]:
# Load Excel file "..\Selection of Prompt Engineering Methods\Hand-Labeled Method and Implementation Considerations.xlsx"
hand_labeled_data = pd.read_excel('..\Selection of Prompt Engineering Methods\Hand-Labeled Method and Implementation Considerations.xlsx')

# Mapping names from this data to technqiue names
# "Few-Shot Learing" -> "manual_few_shot"
# "Chain-of-Thought Prompting" -> "manual_cot"
# "Zero-Shot Chain-of_Thought" -> "zero_shot_cot"
# "Automatic Prompt Engineer" -> "ape_zero_shot_cot"
# "Self-Refine" -> "self_refine"
# "Least-to-Most Prompting" -> "least_to_most"
# "Tree-of-Thought" -> "tree_of_thought"
# Set names in hand_labeled_data to these names
hand_labeled_data['technique_name'] = hand_labeled_data['Prompt Engineering Method'].replace({'Few-Shot Learning': 'manual_few_shot', 'Chain-of-Thought Prompting': 'manual_cot', 'Zero-Shot Chain-of-Thought': 'zero_shot_cot', 'Automatic Prompt Engineer': 'ape_zero_shot_cot', 'Self-Refine': 'self_refine', 'Least-to-Most Prompting': 'least_to_most', 'Tree-of-Thought': 'tree_of_thought'})
# Keep cases where technique_name is not 'Prompt Engineering Method'
hand_labeled_data = hand_labeled_data[hand_labeled_data['technique_name'] != hand_labeled_data['Prompt Engineering Method']]
# Keep cases where technique_name is not missing
hand_labeled_data = hand_labeled_data[hand_labeled_data['technique_name'].notna()]

# Keep columns ss_publication_date and technique_name
hand_labeled_data = hand_labeled_data[['ss_publication_date', 'technique_name']]

# Drop the item with technique_name = 'tree_of_thought' that has the higher ss_publication_date
# Just keep the row with min ss_publication_date for each technique_name
hand_labeled_data = hand_labeled_data.groupby(['technique_name']).min().reset_index()

# Convert ss_publication_date to "YYYY-MM-DD"
# Save datetime version
hand_labeled_data['ss_publication_date_datetime'] = hand_labeled_data['ss_publication_date']
hand_labeled_data['ss_publication_date'] = hand_labeled_data['ss_publication_date'].dt.strftime('%Y-%m-%d')


hand_labeled_data


,technique_name,ss_publication_date,ss_publication_date_datetime
0,ape_zero_shot_cot,2022-11-03,2022-11-03
1,least_to_most,2022-05-21,2022-05-21
2,manual_cot,2022-01-28,2022-01-28
3,manual_few_shot,2020-05-28,2020-05-28
4,self_refine,2023-03-30,2023-03-30
5,tree_of_thought,2023-05-15,2023-05-15
6,zero_shot_cot,2022-05-24,2022-05-24


In [32]:
# check unique values of method in merged_df
print(merged_df['method'].unique())

# Merge on method
merged_df = merged_df.merge(hand_labeled_data, left_on = 'method', right_on = 'technique_name', how='left')

# Added columns - create Method + Publication Date
# Add (Month Year), like "(Jan 2023)" to Method
merged_df['Method + Publication Date'] = merged_df['Method'] + ' (' + merged_df['ss_publication_date_datetime'].dt.strftime('%b %Y') + ')'
# Also create Publication Date column
merged_df['Publication Date'] = merged_df['ss_publication_date_datetime'].dt.strftime('%b %Y')

merged_df


['direct_prompting' 'zero_shot_cot' 'ape_zero_shot_cot' 'least_to_most'
 'manual_few_shot' 'manual_cot' 'tree_of_thought' 'self_refine']


,model_task_method,conversation_number,coherence_1_incoherent_10_very_coherent,compliance_OLD,ease_of_review_1_easy_10_hard,correct,Prediction_Based_On_First_10,Prediction_Based_On_Last_10,Aggregated_Prediction,Prediction_Based_On_First_10_LP,...,accuracy_quality_compliance_adjusted,accuracy_quality_avg_inter_paragraph_cosine_sim,consolidated_num_steps_ideas,consolidated_num_steps_ideas_prompts,consolidated_num_steps_ideas_provided,technique_name,ss_publication_date,ss_publication_date_datetime,Method + Publication Date,Publication Date
0,td3_cw_direct_prompting_responses,1,1.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,...,1.0,0.009602,0,2,NaN,NaN,NaN,NaT,NaN,NaN
1,td3_cw_direct_prompting_responses,2,7.0,0.0,1.0,NaN,NaN,7.0,7.0,NaN,...,1.0,0.586396,0,2,NaN,NaN,NaN,NaT,NaN,NaN
2,td3_cw_direct_prompting_responses,3,1.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,...,1.0,0.372510,0,2,NaN,NaN,NaN,NaT,NaN,NaN
3,td3_cw_direct_prompting_responses,4,10.0,1.0,1.0,NaN,NaN,7.0,7.0,NaN,...,10.0,0.498175,0,2,NaN,NaN,NaN,NaT,NaN,NaN
4,td3_cw_direct_prompting_responses,5,4.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,...,4.0,0.398507,0,2,NaN,NaN,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,gpt4_gsm8k_manual_cot_responses,96,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,1.0,1.000000,3,20,1.0,manual_cot,2022-01-28,2022-01-28,Manual CoT (Jan 2022),Jan 2022
3196,gpt4_gsm8k_manual_cot_responses,97,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,1.0,1.000000,1,20,0.0,manual_cot,2022-01-28,2022-01-28,Manual CoT (Jan 2022),Jan 2022
3197,gpt4_gsm8k_manual_cot_responses,98,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,1.0,1.000000,0,20,0.0,manual_cot,2022-01-28,2022-01-28,Manual CoT (Jan 2022),Jan 2022
3198,gpt4_gsm8k_manual_cot_responses,99,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,1.0,1.000000,1,20,0.0,manual_cot,2022-01-28,2022-01-28,Manual CoT (Jan 2022),Jan 2022


### Create differences variables

In [33]:
# Variables: 
# 'num_linebreaks_prompts': num_linebreaks_prompts,
# 'num_sentences_prompts': num_sentences_prompts,
# 'num_step_i_prompts': num_step_i_prompts,
# 'num_1_dot_etc_prompts': num_1_dot_etc_prompts,
# 'sentence_length_prompts': sentence_length_prompts,
# 'fres_prompts': fres_prompts,
# 'num_linebreaks_provided': num_linebreaks_provided,
# 'num_sentences_provided': num_sentences_provided,
# 'num_step_i_provided': num_step_i_provided,
# 'num_1_dot_etc_provided': num_1_dot_etc_provided
# Loop over variables and create differences variables
comparison_vars = ['num_linebreaks_prompts', 'num_sentences_prompts', 'num_step_i_prompts', 'num_1_dot_etc_prompts', 'sentence_length_prompts', 'fres_prompts', 'num_linebreaks_provided', 'num_sentences_provided', 'num_step_i_provided', 'num_1_dot_etc_provided']
for var in comparison_vars:
    # Create a variable that is the difference between the prompts and responses
    # var + '_diff'
    merged_df[var + '_diff'] = merged_df[var] - merged_df[var.replace('_prompts', '').replace('_provided', '')]


## Combined Output

In [34]:
# Output Combined_Data.xlsx
merged_df.to_excel('Combined_Data.xlsx', index=False)


In [35]:
print(merged_df.columns)


Index(['model_task_method', 'conversation_number',
       'coherence_1_incoherent_10_very_coherent', 'compliance_OLD',
       'ease_of_review_1_easy_10_hard', 'correct',
       'Prediction_Based_On_First_10', 'Prediction_Based_On_Last_10',
       'Aggregated_Prediction', 'Prediction_Based_On_First_10_LP',
       'response_Based_On_First_10_LP', 'Prediction_Based_On_Last_10_LP',
       'response_Based_On_Last_10_LP', 'response_LP',
       'Aggregated_Prediction_LP', 'Prediction_Based_On_First_50_LP',
       'response_Based_On_First_50_LP', 'Prediction_Based_On_Last_50_LP',
       'response_Based_On_Last_50_LP', 'Aggregated_Prediction_50_LP',
       'Prediction_Based_On_random_50_LP_1',
       'response_Based_On_random_50_LP_1',
       'Prediction_Based_On_random_50_LP_2',
       'response_Based_On_random_50_LP_2',
       'Aggregated_Prediction_random_50_LP', 'Unnamed: 0_x', 'response_x',
       'replace_slash_n_slash_n_with_newline_x',
       'replace_slash_n_slash_n_with_newline_values

## Direct Prompting Comparison

In [36]:
# Split dataset
combined_data = pd.read_excel('Combined_Data.xlsx')
no_direct_prompting_data = combined_data[combined_data['method'] != 'direct_prompting']
direct_prompting_data = combined_data[combined_data['method'] == 'direct_prompting']

# Add prefix dp_ to columns in direct_prompting_data
direct_prompting_data = direct_prompting_data.add_prefix('dp_')

# Left join datasets on model = dp_model, task = dp_task
direct_prompting_comparison = no_direct_prompting_data.merge(direct_prompting_data, left_on=['model', 'task', 'conversation_number'], right_on=['dp_model', 'dp_task', 'dp_conversation_number'], how='left')

direct_prompting_comparison


,model_task_method,conversation_number,coherence_1_incoherent_10_very_coherent,compliance_OLD,ease_of_review_1_easy_10_hard,correct,Prediction_Based_On_First_10,Prediction_Based_On_Last_10,Aggregated_Prediction,Prediction_Based_On_First_10_LP,...,dp_num_linebreaks_prompts_diff,dp_num_sentences_prompts_diff,dp_num_step_i_prompts_diff,dp_num_1_dot_etc_prompts_diff,dp_sentence_length_prompts_diff,dp_fres_prompts_diff,dp_num_linebreaks_provided_diff,dp_num_sentences_provided_diff,dp_num_step_i_provided_diff,dp_num_1_dot_etc_provided_diff
0,td3_cw_zero_shot_cot_responses,1,2.0,1.0,2.0,NaN,NaN,1.0,1.0,NaN,...,-1,-2,0,2,-5.971429,12.31,NaN,NaN,NaN,NaN
1,td3_cw_zero_shot_cot_responses,2,8.0,1.0,3.0,NaN,NaN,3.0,3.0,NaN,...,-1,-3,0,2,-9.350000,-1.05,NaN,NaN,NaN,NaN
2,td3_cw_zero_shot_cot_responses,3,8.0,1.0,2.0,NaN,NaN,1.0,1.0,NaN,...,-1,-4,0,2,-4.533333,-6.64,NaN,NaN,NaN,NaN
3,td3_cw_zero_shot_cot_responses,4,9.0,0.0,1.0,NaN,NaN,8.0,8.0,NaN,...,-1,-1,0,2,-9.533333,-9.31,NaN,NaN,NaN,NaN
4,td3_cw_zero_shot_cot_responses,5,1.0,1.0,6.0,NaN,NaN,1.0,1.0,NaN,...,-1,-2,0,2,-7.828571,13.64,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795,gpt4_gsm8k_manual_cot_responses,96,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,-2,0,0,0,NaN,NaN,1.0,3.0,0.0,1.0
2796,gpt4_gsm8k_manual_cot_responses,97,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,1,2,0,0,NaN,NaN,2.0,2.0,0.0,0.0
2797,gpt4_gsm8k_manual_cot_responses,98,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,1,3,0,0,NaN,NaN,3.0,3.0,0.0,0.0
2798,gpt4_gsm8k_manual_cot_responses,99,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,0,2,0,0,NaN,NaN,1.0,1.0,0.0,0.0


In [37]:
list(direct_prompting_comparison.columns)


['model_task_method',
 'conversation_number',
 'coherence_1_incoherent_10_very_coherent',
 'compliance_OLD',
 'ease_of_review_1_easy_10_hard',
 'correct',
 'Prediction_Based_On_First_10',
 'Prediction_Based_On_Last_10',
 'Aggregated_Prediction',
 'Prediction_Based_On_First_10_LP',
 'response_Based_On_First_10_LP',
 'Prediction_Based_On_Last_10_LP',
 'response_Based_On_Last_10_LP',
 'response_LP',
 'Aggregated_Prediction_LP',
 'Prediction_Based_On_First_50_LP',
 'response_Based_On_First_50_LP',
 'Prediction_Based_On_Last_50_LP',
 'response_Based_On_Last_50_LP',
 'Aggregated_Prediction_50_LP',
 'Prediction_Based_On_random_50_LP_1',
 'response_Based_On_random_50_LP_1',
 'Prediction_Based_On_random_50_LP_2',
 'response_Based_On_random_50_LP_2',
 'Aggregated_Prediction_random_50_LP',
 'Unnamed: 0_x',
 'response_x',
 'replace_slash_n_slash_n_with_newline_x',
 'replace_slash_n_slash_n_with_newline_values_x',
 'replace_slash_n_with_newline_x',
 'replace_slash_n_with_newline_values_x',
 'avg_co

In [38]:
# Save direct_prompting_comparison to Excel
direct_prompting_comparison.to_excel('direct_prompting_comparison.xlsx', index=False)
